In [235]:
%reset -f

In [236]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import plotly.graph_objs as go
import plotly.io as pio
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks, detrend
from scipy.signal import correlate
from scipy.stats import pearsonr
from plotly.subplots import make_subplots
import csv
from biosppy.signals import ecg

In [237]:
# Sets figure show to pop on default browser
pio.renderers.default = 'browser'

In [238]:
df = pd.read_csv('ali_validation/Trial 11/30s_b_40s_h_1m_b_4x_alivalidation11_masimocomp.csv')
raw_redlight = df['Red Light'].values
raw_infaredlight = df['IR'].values
timestamps = df['Time Stamp'].values

In [239]:
# Convert PPG time stamps to float and then to seconds from start
df["time_ms"] = df["Time Stamp"].astype(float)
df["time_sec"] = (df["time_ms"] - df["time_ms"].iloc[0]) / 1000.0

In [240]:
t_raw = df["time_sec"].to_numpy()
red_raw = df["Red Light"].to_numpy()
ir_raw = df["IR"].to_numpy()

In [241]:
# Interpolate to uniform sampling of 250Hz
desired_fs = 250  # Hz
dt = 1 / desired_fs
t_uniform = np.arange(t_raw[0], t_raw[-1], dt)

red_interp = interp1d(t_raw, red_raw, kind='linear', fill_value="extrapolate")
ir_interp = interp1d(t_raw, ir_raw, kind='linear', fill_value="extrapolate")

red = red_interp(t_uniform)
ir = ir_interp(t_uniform)

In [242]:
# Apply 1D Gaussian smoothing, keep sigma 6 for optimal smoothing
red_smoothed = gaussian_filter1d(red, sigma=6)
ir_smoothed = gaussian_filter1d(ir, sigma=6)

In [243]:
#Detrend the smoothed signals
red_detrended = detrend(red_smoothed)
ir_detrended = detrend(ir_smoothed)

# Peak/Trough Detection with Prominence and Amplitude Filtering, the prominence has been optimized to 280 for infared signal
def detect_peaks_and_troughs_ir(signal, min_thresh=70000, max_thresh=150000, prominence=350):
    # Detect peaks
    peaks, _ = find_peaks(signal, prominence=prominence)
    peaks = [idx for idx in peaks if min_thresh < signal[idx] < max_thresh]

    # Detect troughs by inverting the signal
    troughs, _ = find_peaks(-signal, prominence=prominence)
    troughs = [idx for idx in troughs if min_thresh < signal[idx] < max_thresh]

    return np.array(peaks), np.array(troughs)

# Peak/Trough Detection with Prominence and Amplitude Filtering, the prominence has been optimized to 175 for red light signal
def detect_peaks_and_troughs_red (signal, min_thresh=70000, max_thresh=150000, prominence=200):
    # Detect peaks
    peaks, _ = find_peaks(signal, prominence=prominence)
    peaks = [idx for idx in peaks if min_thresh < signal[idx] < max_thresh]

    # Detect troughs by inverting the signal
    troughs, _ = find_peaks(-signal, prominence=prominence)
    troughs = [idx for idx in troughs if min_thresh < signal[idx] < max_thresh]

    return np.array(peaks), np.array(troughs)

# === Apply detection ===
red_peaks_idx, red_troughs_idx = detect_peaks_and_troughs_red (red_smoothed)
ir_peaks_idx, ir_troughs_idx = detect_peaks_and_troughs_ir (ir_smoothed)

In [244]:
# Plot raw Red and IR with filtered peaks/troughs
fig = go.Figure()

# Red
fig.add_trace(go.Scatter(x=t_uniform, y=red, mode='lines', name='Red Raw', line=dict(color='red')))
fig.add_trace(go.Scatter(x=t_uniform[red_peaks_idx], y=red[red_peaks_idx], mode='markers',
                         name='Red Peaks', marker=dict(symbol='triangle-up', color='black')))
fig.add_trace(go.Scatter(x=t_uniform[red_troughs_idx], y=red[red_troughs_idx], mode='markers',
                         name='Red Troughs', marker=dict(symbol='triangle-down', color='black')))

# IR
fig.add_trace(go.Scatter(x=t_uniform, y=ir, mode='lines', name='IR Raw', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=t_uniform[ir_peaks_idx], y=ir[ir_peaks_idx], mode='markers',
                         name='IR Peaks', marker=dict(symbol='triangle-up', color='blue')))
fig.add_trace(go.Scatter(x=t_uniform[ir_troughs_idx], y=ir[ir_troughs_idx], mode='markers',
                         name='IR Troughs', marker=dict(symbol='triangle-down', color='blue')))

fig.update_layout(
    title="Raw Signals with Threshold-Filtered Peaks and Troughs",
    xaxis_title="Time (s)",
    yaxis_title="Amplitude",
    hovermode="x unified"
)
fig.show()

In [245]:
# Heart Rate Calculation from IR Peaks
ir_peak_times = t_uniform[ir_peaks_idx]
intervals = np.diff(ir_peak_times)
bpm = 60 / intervals
bpm_time = ir_peak_times[:-1]

# Clean BPM values by thresholding
bpm_clean = np.where((bpm < 40) | (bpm > 150), np.nan, bpm)
bpm_clean = pd.Series(bpm_clean).interpolate().to_numpy()

# Apply Moving Average Smoothing
window = 20  # adjust this to control smoothing
bpm_smooth = pd.Series(bpm_clean).rolling(window=window, center=True, min_periods=1).median().to_numpy()

# Plot Heart Rate Over Time
hr_fig = go.Figure()
hr_fig.add_trace(go.Scatter(x=bpm_time, y=bpm_clean, mode='lines+markers',
                            name='Raw BPM', line=dict(color='lightgreen')))
hr_fig.add_trace(go.Scatter(x=bpm_time, y=bpm_smooth, mode='lines',
                            name='Smoothed BPM (Moving Avg)', line=dict(color='green')))
hr_fig.update_layout(
    title="Heart Rate Over Time (from IR Peaks, with Smoothing)",
    xaxis_title="Time (s)",
    yaxis_title="Beats Per Minute (BPM)",
    hovermode="x unified"
)
hr_fig.show()

In [246]:
# Match IR and Red Peaks (50 ms window)
matched_ir_peaks = []
matched_red_peaks = []
window = int(0.05 * desired_fs)  # 50 ms window if fs = 250 Hz

for ir_idx in ir_peaks_idx:
    # Search for closest red peak within window
    candidates = red_peaks_idx[(red_peaks_idx >= ir_idx - window) & (red_peaks_idx <= ir_idx + window)]
    if len(candidates) > 0:
        closest_red = candidates[np.argmin(np.abs(candidates - ir_idx))]
        matched_ir_peaks.append(ir_idx)
        matched_red_peaks.append(closest_red)

# SpO₂ Calculation
spo2_list = []
spo2_time = []

for i in range(len(matched_ir_peaks) - 1):
    ir_start = matched_ir_peaks[i]
    ir_end = matched_ir_peaks[i + 1]
    red_idx = matched_red_peaks[i]

    red_seg = red[red_idx:ir_end]
    ir_seg = ir[ir_start:ir_end]

    if len(red_seg) < 2 or len(ir_seg) < 2:
        continue

    ac_red = np.max(red_seg) - np.min(red_seg)
    dc_red = np.mean(red_seg)
    ac_ir = np.max(ir_seg) - np.min(ir_seg)
    dc_ir = np.mean(ir_seg)

    if dc_red == 0 or dc_ir == 0:
        continue

    R = (ac_red / dc_red) / (ac_ir / dc_ir)
    spo2 = 110 - 25 * R

    if 70 <= spo2 <= 110:
        spo2_list.append(spo2)
        spo2_time.append(t_uniform[ir_start])


# Plot Smoothed SpO₂
fig_spo2 = go.Figure()
fig_spo2.add_trace(go.Scatter(x=spo2_time, y=spo2_list, mode='lines+markers', name='Filtered SpO₂'))
fig_spo2.update_layout(title="SpO₂ Over Time", xaxis_title="Time (s)", yaxis_title="SpO₂ (%)")
fig_spo2.show()

In [247]:
# Match IR and Red Peaks (50 ms window)
matched_ir_peaks = []
matched_red_peaks = []
window = int(0.05 * desired_fs)  # 50 ms window if fs = 250 Hz

for ir_idx in ir_peaks_idx:
    # Search for closest red peak within window
    candidates = red_peaks_idx[(red_peaks_idx >= ir_idx - window) & (red_peaks_idx <= ir_idx + window)]
    if len(candidates) > 0:
        closest_red = candidates[np.argmin(np.abs(candidates - ir_idx))]
        matched_ir_peaks.append(ir_idx)
        matched_red_peaks.append(closest_red)

# SpO₂ Calculation
spo2_list = []
spo2_time = []

for i in range(len(matched_ir_peaks) - 1):
    ir_start = matched_ir_peaks[i]
    ir_end = matched_ir_peaks[i + 1]
    red_idx = matched_red_peaks[i]

    red_seg = red[red_idx:ir_end]
    ir_seg = ir[ir_start:ir_end]

    if len(red_seg) < 2 or len(ir_seg) < 2:
        continue

    ac_red = np.max(red_seg) - np.min(red_seg)
    dc_red = np.mean(red_seg)
    ac_ir = np.max(ir_seg) - np.min(ir_seg)
    dc_ir = np.mean(ir_seg)

    if dc_red == 0 or dc_ir == 0:
        continue

    R = (ac_red / dc_red) / (ac_ir / dc_ir)
    spo2 = 110 - 25 * R

    if 70 <= spo2 <= 110:
        spo2_list.append(spo2)
        spo2_time.append(t_uniform[ir_start])

# Round and Smooth Final SpO₂ Values

spo2_df = pd.DataFrame({
    'Time': spo2_time,
    'SpO2': spo2_list
})

# Round to nearest integer
spo2_df['SpO2'] = spo2_df['SpO2'].round()

# Rolling median smoothing with window=5
spo2_df['SpO2'] = spo2_df['SpO2'].rolling(window=20, center=True, min_periods=1).median()

# Plot Smoothed SpO₂
fig_spo2 = go.Figure()
fig_spo2.add_trace(go.Scatter(x=spo2_df['Time'], y=spo2_df['SpO2'], mode='lines+markers', name='Filtered SpO₂'))
fig_spo2.update_layout(title="SpO₂ Over Time", xaxis_title="Time (s)", yaxis_title="SpO₂ (%)")
fig_spo2.show()

In [250]:
# with rounding and moving median SpO2 compared with masimo
# Plots data from masimo ppg and plots it against analyzed ppg data
masimo_df = pd.read_csv("ali_validation/Trial 11/30s_b_40s_h_30s_h_4x_alivalidation11_masimodevice.csv")

# Convert to numeric
masimo_df['O2 Saturation'] = pd.to_numeric(masimo_df['O2 Saturation'], errors='coerce')
masimo_df['Pulse Rate'] = pd.to_numeric(masimo_df['Pulse Rate'], errors='coerce')

# Drop NaNs and reset index
masimo_df = masimo_df[['O2 Saturation', 'Pulse Rate']].dropna().reset_index(drop=True)

# Assume 1 Hz sampling rate (1 sample per second)
masimo_time = masimo_df.index.to_numpy()
masimo_spo2 = masimo_df['O2 Saturation'].to_numpy()
masimo_hr = masimo_df['Pulse Rate'].to_numpy()

# Shift masimo time to align start times (e.g., -3 seconds if Masimo started 3 seconds earlier)
masimo_offset = -5  # adjust based on your known delay
masimo_time_shifted = masimo_time + masimo_offset

# Plot SpO₂ Comparison
fig_spo2 = go.Figure()
fig_spo2.add_trace(go.Scatter(x=spo2_df['Time'], y=spo2_df['SpO2'], mode='lines+markers',
                              name='Vena Vitals SpO₂', line=dict(color='blue')))
fig_spo2.add_trace(go.Scatter(x=masimo_time_shifted, y=masimo_spo2, mode='lines+markers',
                              name='Masimo SpO₂', line=dict(color='orange')))
fig_spo2.update_layout(title="SpO₂ Comparison: Vena Vitals vs Masimo",
                       xaxis_title="Time (s)", yaxis_title="SpO₂ (%)", hovermode="x unified")
fig_spo2.show()

# Plot Heart Rate Comparison
fig_hr = go.Figure()
fig_hr.add_trace(go.Scatter(x=bpm_time, y=bpm_smooth, mode='lines+markers',
                            name='Vena Vitals (BPM)', line=dict(color='green')))
fig_hr.add_trace(go.Scatter(x=masimo_time_shifted, y=masimo_hr, mode='lines+markers',
                            name='Masimo HR (BPM)', line=dict(color='red')))
fig_hr.update_layout(title="Pulse Rate Comparison: Vena Vitals vs Masimo",
                     xaxis_title="Time (s)", yaxis_title="BPM", hovermode="x unified")
fig_hr.show()

In [249]:
import pandas as pd
import numpy as np

# === Load and preprocess Masimo data ===
masimo_df = pd.read_csv("ali_validation/Trial 11/30s_b_40s_h_30s_h_4x_alivalidation11_masimodevice.csv")
masimo_df['O2 Saturation'] = pd.to_numeric(masimo_df['O2 Saturation'], errors='coerce')
masimo_df['Pulse Rate'] = pd.to_numeric(masimo_df['Pulse Rate'], errors='coerce')
masimo_df = masimo_df[['O2 Saturation', 'Pulse Rate']].dropna().reset_index(drop=True)

# Create time array and apply 5-second offset
masimo_time = masimo_df.index.to_numpy()
masimo_time_shifted = masimo_time - 5

# Mask out negative times after offset
valid_mask = masimo_time_shifted >= 0
masimo_spo2 = masimo_df['O2 Saturation'].to_numpy()[valid_mask]
masimo_hr = masimo_df['Pulse Rate'].to_numpy()[valid_mask]

# === Align lengths by trimming to the shortest length ===
min_length_spo2 = min(len(spo2_df['SpO2']), len(masimo_spo2))
min_length_hr = min(len(bpm_smooth), len(masimo_hr))

# === SpO2 Deviation Metrics ===
spo2_diff = np.abs(spo2_df['SpO2'][:min_length_spo2].to_numpy() - masimo_spo2[:min_length_spo2])
spo2_mad = np.mean(spo2_diff)
spo2_std = np.std(spo2_df['SpO2'][:min_length_spo2].to_numpy() - masimo_spo2[:min_length_spo2])
spo2_meandev = np.mean(spo2_df['SpO2'][:min_length_spo2].to_numpy() - masimo_spo2[:min_length_spo2])

# === Heart Rate Deviation Metrics ===
hr_diff = np.abs(bpm_smooth[:min_length_hr] - masimo_hr[:min_length_hr])
hr_mad = np.mean(hr_diff)
hr_std = np.std(bpm_smooth[:min_length_hr] - masimo_hr[:min_length_hr])
hr_meandev = np.mean(bpm_smooth[:min_length_hr] - masimo_hr[:min_length_hr])

# === RMSD Function ===
def rmsd(y1, y2):
    return np.sqrt(np.mean((np.array(y1) - np.array(y2)) ** 2))

rmd_spo2 = rmsd(spo2_df['SpO2'][:min_length_spo2].to_numpy(), masimo_spo2[:min_length_spo2])
rmd_hr = rmsd(bpm_smooth[:min_length_hr], masimo_hr[:min_length_hr])

# === Print Results ===
print(f"SpO₂ Mean Absolute Deviation: {spo2_mad:.2f}")
print(f"SpO₂ Standard Deviation: {spo2_std:.2f}")
print(f"SpO₂ Mean Deviation: {spo2_meandev:.2f}")
print(f"Heart Rate Mean Absolute Deviation: {hr_mad:.2f}")
print(f"Heart Rate Standard Deviation: {hr_std:.2f}")
print(f"Heart Rate Mean Deviation: {hr_meandev:.2f}")
print(f"Root Mean Square Difference of SpO2: {rmd_spo2:.2f}")
print(f"Root Mean Square Difference of Heart Rate: {rmd_hr:.2f}")

SpO₂ Mean Absolute Deviation: 3.41
SpO₂ Standard Deviation: 4.24
SpO₂ Mean Deviation: 2.00
Heart Rate Mean Absolute Deviation: 3.41
Heart Rate Standard Deviation: 4.11
Heart Rate Mean Deviation: -0.13
Root Mean Square Difference of SpO2: 4.69
Root Mean Square Difference of Heart Rate: 4.11
